<img src='./img/EU-Copernicus-EUM_3Logos.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='50%'/>

<hr>

# MTG FCI and S3 for fires: visualising FCI L1c RGBs, FIR Active Fire Monitoring product and S3 NRT Fire detections

Author: Andrea Meraner (EUMETSAT). Containing parts of notebooks originally developed by Dominika Czyzewska (EUM) and Noemi Fazzini (MEEO)

### Data used

|                    Product Description                     | Data Store collection ID |                     Product Navigator                      |
|:----------------------------------------------------------:|:------------------------:|:----------------------------------------------------------:|
|      Active Fire Monitoring (netCDF) - MTG - 0 degree      |     EO:EUM:DAT:0682      | [link](https://vdata.eumetsat.int/product/EO:EUM:DAT:0682) |
| FCI Level 1c Normal Resolution Image Data - MTG - 0 degree |     EO:EUM:DAT:0662      | [link](https://vdata.eumetsat.int/product/EO:EUM:DAT:0662) |
|  FCI Level 1c High Resolution Image Data - MTG - 0 degree  |     EO:EUM:DAT:0665      | [link](https://vdata.eumetsat.int/product/EO:EUM:DAT:0665) |
| SLSTR Level 2 Fire Radiative Power in NRT - Sentinel-3  | EO:EUM:DAT:0417 | [link](https://navigator.eumetsat.int/product/EO:EUM:DAT:0207) |


This notebook demonstrates how to access, load, and visualise the [Active Fire Monitoring product (FIR)](https://vdata.eumetsat.int/product/EO:EUM:DAT:0682) product derived from Meteosat Third Generation (MTG) geostationary imager data. The FIR product detects and characterises active fires within pixels using the FCI IR-3.8 μm channel, which is highly sensitive to fire hotspots.

In addition, [FCI Level 1c Normal Resolution Image Data](https://vdata.eumetsat.int/product/EO:EUM:DAT:0662) consists of a set of files that contain the level 1c science data rectified to a reference grid together with the auxiliary data associated with the processing configuration and the quality assessment of the dataset. [FCI Level 1c High Resolution Image Data](https://vdata.eumetsat.int/product/EO:EUM:DAT:0665) contains 4 channels (VIS0.6, NIR2.2, IR3.8, and IR10.5) at an additional high resolution (500m for the VISNIR channels and 1km for the IR channels).

At the end, we also load and compare Sentinel-3 NRT Fire product detections with the FCI imagery.

As a case study, this notebook explores the Iberian Pennisula wildfires of summer 2025, combining FIR datasets with True Colour RGB composites to illustrate fire monitoring capabilities under varying conditions of cloud cover and surface features.



#### Load required libraries

In [ ]:
import datetime
import os
import numpy as np
from pathlib import Path
import glob

from satpy.scene import Scene
from satpy import find_files_and_readers
from satpy.writers import get_enhanced_image
from pyresample import create_area_def

import warnings
from cartopy.io import DownloadWarning
import xarray as xr

from download_from_archive import download_from_eumdac
import credentials

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib import cm
import cartopy.crs as ccrs


warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings("ignore", category=DownloadWarning)


#### Set the bounding box of the area of interest and the start-end datetimes to analyse

<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
For the notebook to work correctly, use a 10 minutes start-end time interval (the FCI products are available every 10 minutes).
</div>

In [ ]:
latmin = 37.0000
latmax= 43.7914
lonmin= -9.5600
lonmax= 3.3322

start_time = "2025-08-18T16:00:00"
end_time = "2025-08-18T16:10:00"
# start_time = "2025-08-18T22:20:00"
# end_time = "2025-08-18T22:30:00"

#### Set the resolution of the target analysis area definition and create the according pyresample AreaDefinition object

In [ ]:
resolution = 500. #m

In [ ]:
def get_local_mercator_area(target_proj, area_extent, resolution):
    x_min, y_min = target_proj.transform_point(area_extent[0], area_extent[1], ccrs.PlateCarree()) # compute coordinates in target projection
    x_max, y_max = target_proj.transform_point(area_extent[2], area_extent[3], ccrs.PlateCarree())
    shape = [int((y_max - y_min) / resolution[0]), int((x_max - x_min) / resolution[1])] # compute the shape of the area in pixels
    adef = create_area_def('local_mercator_area', target_proj, area_extent=area_extent, units='degrees', shape=shape)
    return adef

target_proj = ccrs.Mercator()
adef = get_local_mercator_area(target_proj, [lonmin, latmin, lonmax, latmax], [resolution, resolution])
print(adef)


## Download FIR products from the Data Store through the EUMDAC API

<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
Before running this section, remember to edit the credentials.py file with your personal API credentials for EUMDAC. You can find your personal API credentials here: <a href="https://api.eumetsat.int/api-key/">https://api.eumetsat.int/api-key/</a>
</div>

The following cell calls the `download_from_archive.download_from_eumdac` that is a custom wrapper around the EUMDAC API. The following cell shows the function docstring giving some extra information.

FYI: MTG products follow the WMO file naming convention. You can find more information about this in our User Portal guide: <a href="https://user.eumetsat.int/resources/user-guides/mtg-fci-level-1c-data-guide#ID-Naming-convention">https://user.eumetsat.int/resources/user-guides/mtg-fci-level-1c-data-guide#ID-Naming-convention</a>
</div>

In [ ]:
help(download_from_eumdac)

In [ ]:
output_folder = "products/"

collection_ids = ["EO:EUM:DAT:0682"]
file_endings = ['.nc']

download_from_eumdac(start_time, end_time, collection_ids, output_folder, credentials.EUMDAC_CONSUMER_KEY,
                     credentials.EUMDAC_CONSUMER_SECRET, file_endings=file_endings)

## FIR Product Processing

Let us open the downloaded Active Fire Monitoring product using `Scene` from `satpy` library, and inspect it using the function `available_dataset_names`.

The product provides two main datasets at 2 km nadir resolution:
* `Fire Probability` – the likelihood of fire presence per pixel.
* `Fire Result` – a categorical indicator derived from fire probability.

In addition, quality indicators (`product_completeness`, `product_quality`, `product_timeliness`) are included for each repeat cycle to support data interpretation. The FIR product is currently based on the heritage SEVIRI algorithm, ensuring continuity with existing fire detection services. While it does not yet exploit the full spatial resolution and additional channels of the FCI, future product updates will enhance detection accuracy and resolution.

The reader used in this case is `fci_l2_nc`.

In [ ]:
fir_filenames = find_files_and_readers(base_dir=output_folder, reader="fci_l2_nc", start_time=datetime.datetime.strptime(start_time, "%Y-%m-%dT%H:%M:%S"), end_time=datetime.datetime.strptime(end_time, "%Y-%m-%dT%H:%M:%S"))

fir_scn = Scene(filenames=fir_filenames)
print(fir_scn.available_dataset_names())  # Prints list of available datasets

This cell performs the following steps:
1. Load the fire probability dataset (`fire_probability`) from the Satpy Scene and ensure it is oriented upright.
2. Resample the dataset to the defined Area of Interest (AOI).
4. Filter pixels with probability above a threshold (e.g., 0.5) to identify potential fire locations.

In [ ]:
dataset = "fire_probability"  # Dataset to load. Other option is "fire_result"

# Load dataset. The upper_right_corner keyword is needed to flip the dataset to "upright" projection
fir_scn.load([dataset], upper_right_corner='NE')

scn_resample=fir_scn.resample(adef)

# Extract dataset from Scene object
fir_prob = scn_resample[dataset]

# filter fires above a probability threshold. Note that the probability range is 0-1
prob_threshold = 0.5
fires_above_th = np.where(fir_prob > prob_threshold)
print(f"Number of potential fire pixels in target grid with probability above 50%: {len(fires_above_th[0])}")

# The following line obtains the lon-lat of the filtered pixels. We continue to work on the 2d-array so we don't need it.
lon_filtered, lat_filtered = adef.get_lonlat_from_array_coordinates(fires_above_th[1], fires_above_th[0])
lon_filtered, lat_filtered

Now plot the dataset using `matplotlib.pyplot` and `cartopy`.

In [ ]:
crs = adef.to_cartopy_crs()  # Convert area definition to cartopy CRS object
fig, ax = plt.subplots(subplot_kw={'projection': crs})

ax.coastlines(resolution='50m')

im = ax.imshow(fir_prob.values*100, transform=crs, cmap='RdYlBu_r', extent=crs.bounds, origin='upper', vmin=0, vmax=100, interpolation='none')
plt.axis('off')
cbar = plt.colorbar(im, orientation='horizontal', aspect=25, shrink=0.72, pad=0.02)
cbar.ax.set_xlabel('FCI FIR fire probability (%)')
plt.tight_layout()
plt.show()

This cell repeats the operation for the the fire detection result dataset (`fire_result`)

In [ ]:
dataset = "fire_result"  # Dataset to load. 
# Load dataset. The upper_right_corner keyword is needed to flip the dataset to "upright" projection
fir_scn.load([dataset], upper_right_corner='NE')
scn_resample=fir_scn.resample(adef)

fir_res = scn_resample[dataset]

Now plot the result dataset using `matplotlib.pyplot` and `cartopy`. In this case the labels are categorical (discrete) classes, not continuous values. Each pixel in the map belongs to exactly one of the categories.

Since they are discrete classification results, they are represented with a ListedColormap (one distinct color per class) and BoundaryNorm to map the integer values to the correct color interval.

In [ ]:
result_labels = {
    0: 'No fire',
    1: 'Fire low confidence',
    2: 'Fire medium confidence',
    3: 'Fire high confidence',
    4: 'Missing-undefined'
}

colors = ['lightgray', 'yellow', 'orange', 'red', 'white']
cmap = ListedColormap(colors)
bounds = [0, 1, 2, 3, 4, 5]  # for 5 classes
norm = BoundaryNorm(bounds, cmap.N)

# Plot
crs = adef.to_cartopy_crs()
fig, ax = plt.subplots(subplot_kw={'projection': crs})

ax.coastlines(resolution='50m')
im = ax.imshow(fir_res, transform=crs, cmap=cmap, norm=norm, extent=crs.bounds, origin='upper', interpolation='nearest')

plt.axis('off')
cbar = plt.colorbar(im, orientation='horizontal', aspect=25, shrink=0.72, pad=0.02, ticks=[0.5, 1.5, 2.5, 3.5, 4.5])
labels = [result_labels[i] for i in range(5)]
cbar.ax.set_xticklabels(labels)
cbar.ax.set_xticklabels(labels, rotation=45, ha='right')
cbar.ax.set_xlabel('Fire Detection Result')

plt.tight_layout()
plt.show()

## 3 - Load and visualise FCI Level 1c Data

Now let's download the FCI L1c data for the same area and time as the FIR product. 

The FCI L1c data is split in 40 chunks, that when stacked together, compose a full-disc image (see [this guide for a plot of the chunks location](https://user.eumetsat.int/resources/user-guides/mtg-data-access-guide#ID-Download-chunks-of-FCI-products-based-on-coverage)). Adding the `fci_l1c_chunks_lonlat_bbox` argument lets the code only download chunks that intersect the bounding box.

In [ ]:
# geographical bounds of search area (used for FCI L1c chunks selection)
fci_l1c_chunks_lonlat_bbox = [lonmin, latmin, lonmax, latmax]

collection_ids = ["EO:EUM:DAT:0662", "EO:EUM:DAT:0665"]

download_from_eumdac(start_time, end_time, collection_ids, output_folder,
                     credentials.EUMDAC_CONSUMER_KEY, credentials.EUMDAC_CONSUMER_SECRET, file_endings=file_endings,
                     fci_l1c_chunks_lonlat_bbox=fci_l1c_chunks_lonlat_bbox)


You can now use the `Scene` constructor from the [satpy](https://satpy.readthedocs.io/en/stable/index.html) library to load the data files. Once loaded, a `Scene` object represents a single geographic region of data.

The Satpy Scene accepts both FDHSI and HRFI data chunks at the same time, and will automatically pick the channels with the best available resolution by default.

In [ ]:
fci_files = find_files_and_readers(base_dir=output_folder, reader='fci_l1c_nc', start_time=datetime.datetime.strptime(start_time, "%Y-%m-%dT%H:%M:%S"), end_time=datetime.datetime.strptime(end_time, "%Y-%m-%dT%H:%M:%S"))
fci_l1_scn = Scene(filenames=fci_files)

Here you will be able to select the RGB composite you want to display. You can browse the pre-defined composites using the according function

In [ ]:
fci_l1_scn.available_composite_names()


We choose to display a True Color RBG:

In [ ]:
composite1 = 'true_color'
fci_l1_scn.load([composite1], upper_right_corner='NE', pad_data=False)

Next, you can use the area definition above in order to resample the loaded Scene object. You can use the function `resample()` to do so.
Let us also save in `xr_color` the image for future plotting.

In [ ]:
scn_resampled_color = fci_l1_scn.resample(adef)
xr_color = get_enhanced_image(scn_resampled_color["true_color"])
composite1_img = np.moveaxis(xr_color.data.to_numpy(), 0, -1)

You can now print the metadata of the loaded image

In [ ]:
print(scn_resampled_color)

Afterwards, you can visualize the resampled `true_color` RGB with the function `show()`.

In [ ]:
scn_resampled_color.show(composite1)

The same thing can be done with channel/RGB composite `fire_temperature`. More informations can be found [here](https://user.eumetsat.int/resources/user-guides/fire-temperature-rgb-quick-guide).

In [ ]:
composite2='fire_temperature'
fci_l1_scn.load([composite2], upper_right_corner='NE', pad_data=False)

scn_resampled_temp = fci_l1_scn.resample(adef)
xr_temp = get_enhanced_image(scn_resampled_temp["fire_temperature"])
composite2_img = np.moveaxis(xr_temp.data.to_numpy(), 0, -1)

scn_resampled_temp.show(composite2)

## 4 - Compare FIR data with RGB composites

Plots a 3-panel map for comparison of different data: fire probability heatmap, fire classification categories, and a normalized Satpy RGB scene.

In [ ]:
%matplotlib widget
# Define CRS
crs = adef.to_cartopy_crs()

# Prepare colormap
result_labels = {
    0: 'No fire',
    1: 'Fire low confidence',
    2: 'Fire medium confidence',
    3: 'Fire high confidence',
    4: 'Missing-undefined'
}
colors = ['lightgray', 'yellow', 'orange', 'red', 'white']
cmap = ListedColormap(colors)
bounds = [0, 1, 2, 3, 4, 5]
norm = BoundaryNorm(bounds, cmap.N)

fig, axes = plt.subplots(1, 4, subplot_kw={'projection': crs}, figsize=(18, 6))

# Fire probability map 
ax = axes[0]
ax.coastlines(resolution='50m')
im1 = ax.imshow(fir_prob.values*100, transform=crs, cmap='RdYlBu_r',
                extent=crs.bounds, origin='upper',
                vmin=0, vmax=100, interpolation='none')
ax.set_title("MTG FIR Fire Probability (%)")

# Fire classification result 
ax = axes[1]
ax.coastlines(resolution='50m')
im2 = ax.imshow(fir_res, transform=crs, cmap=cmap, norm=norm,
                extent=crs.bounds, origin='upper', interpolation='nearest')
ax.set_title("MTG FIR Fire Detection Result")

# Satpy scenes 
ax = axes[2]
ax.imshow(composite1_img, transform=crs, origin='upper')
ax.set_title(f"MTG {composite1} Image")

ax = axes[3]
ax.imshow(composite2_img, transform=crs, origin='upper')
ax.set_title(f"MTG {composite2} Image")

plt.tight_layout()
plt.show()


## Load and visualise S3 NRT FRP product

The [Copernicus Sentinel-3 SLSTR Near-Real-Time Fire Radiative Power (FRP)](https://navigator.eumetsat.int/product/EO:EUM:DAT:0207) product identifies the location and quantifies the radiative power of any hotspot present on land and ocean, that radiates a heating signal within a pixel size of 1 km<sup>2</sup>.

All  hotspots are identified and characterised within three hours from SLSTR observation sensing time.
The current version of the Near-Real-Time (NRT) S3 FRP processor (Collection 3 available since 4 July 2024) is applicable day and night (over land), and night (over waters for offshore gas flares). The NRT S3 FRP product is pre-operational after having reached lately a high level of quality and maturity, a comprehensive global validation, and positive feedback by experts and users.


As with FCI, let's download the S3 data through the EUMDAC wrapper. The `search_bbox` argument allows the filtering of the LEO granules according to our AOI. With LEO, it

In [ ]:
search_bbox = ",".join([str(lonmin), str(latmin), str(lonmax), str(latmax)])
collection_ids = ["EO:EUM:DAT:0417"]
download_from_eumdac(start_time, end_time, collection_ids, output_folder,
                     credentials.EUMDAC_CONSUMER_KEY, credentials.EUMDAC_CONSUMER_SECRET, file_endings=file_endings,
                     search_bbox=search_bbox)

<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
The call above, with the default dates, may not have downloaded any data. Why is that the case? And how can we solve this?
</div>

The first step is to load the data file with xarray's `xr.open_dataset()` function.

Once the data file is loaded, we see that the data file has three dimensions: `columns`, `fires` and `rows`. The data and additional information, such as quality flags or latitude and longitude information, is stored as data variables.

There are 2 variables of interest:
- `FRP_MWIR` - Fire Radiative Power computed from MWIR channel (3.7 um) [MW]
- `FRP_SWIR` - Fire Radiative Power computed from SWIR channel (2.25 um) [MW]

The channels differ in the type of fire they can detect. The `FRP_MWIR` channel detects hotspots with a temperature lower than 1100 Kelvin (typically wildfires), whereas the `FRP_SWIR` channel detects hotspots with higher temperatures above 1100 Kelvin (intense wildfires and gas flares). Both channels can be processes as shown below, in this notebook we will focus only on `FRP_MWIR`.

FRP products mainly use MWIR during the day due to its reliability, while at night, combining MWIR and SWIR helps detect a wider range of fire intensities. SWIR is not yet used during daytime. We select the `FRP_MWIR1km_standard.nc` file to analyse the MWIR detections.

In [ ]:
# You can change the folder below if you like, the preselected one is recommended as it contains many fires
data_folder = 'S3B_SL_2_FRP____20250818T222627_20250818T223127_20250819T004819_0299_110_101______MAR_O_NR_003.SEN3'

frp_xr = xr.open_dataset(f"./products/{data_folder}/FRP_MWIR1km_standard.nc", engine='netcdf4')
frp_xr

In [ ]:
lat_frp = frp_xr['latitude']
lon_frp = frp_xr['longitude']
frp_val = frp_xr['FRP_MWIR']

lat_frp, lon_frp, frp_val

In the following cell, feel free to extract some interesting statistics from the variables.

In [ ]:
# your code to extract statistics

Now, let's plot the S3 detections on top of FCI data! If the selected S3 granule times do not correspond to the FCI times selected previously, just change the times at the top of the notebook and re-run all cells until here.

You can uncomment the FCI plot you would like to add below the S3 granules.

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={'projection': crs}, figsize=(18, 6))
ax.coastlines(resolution='50m')



# ax.imshow(fir_prob.values*100, transform=crs, cmap='RdYlBu_r', extent=crs.bounds, origin='upper', vmin=0, vmax=100, interpolation='none')  # fire probability map
# ax.imshow(fir_res, transform=crs, cmap=cmap, norm=norm, extent=crs.bounds, origin='upper', interpolation='nearest')  # fire classification result
# ax.imshow(composite1_img, transform=crs, origin='upper', extent=crs.bounds)  # satpy composite 1
ax.imshow(composite2_img, transform=crs, origin='upper', extent=crs.bounds)  # satpy composite 2


scatter = ax.scatter(
    lon_frp.values, lat_frp.values,
    c=frp_val,
    cmap='YlOrRd',
    s=30,  # size of markers
    edgecolor='black',  # optional: add a black border around markers
    transform=ccrs.PlateCarree(),
    vmin=0, vmax=600
)

colormap = cm.ScalarMappable(cmap='YlOrRd')
colormap.set_clim(0, 600)
cbar = plt.colorbar(colormap, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('S3 FRP Value [MW]')

<hr>

References:
* https://gitlab.eumetsat.int/eumetlab/data-services/eumdac_data_store/-/blob/master/1_5_MTG_FCI_data_access.ipynb
* https://www.eumetrain.org/resources/andrea-meraner-eumetsat-fci4fires-detecting-and-visualising-wildfires-mtg-fci (slides)
* https://user.eumetsat.int/resources/user-guides/mtg-fci-l2-fir-data-guide#ID-Data-loading-analysis-processing-and-visualisation-tools

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE.TXT">GPL-3.0-or-later</a> </p>